# Multi-Free Field Analysis Example using DAPI

This example shows how to run OpenSeesMP in DesignSafe from a jupyter notebook using the DesignSafe API (dapi).

A set of four 1D profiles is analyzed using OpenSeesMP.

<img src = "multi-freeField.png"  height="400" width="400" align = "center">

# Setup DAPI and start OpenSeesMP job

In [ ]:
# Install required packages
!python -m pip install --upgrade numpy
!pip install -e ../../

### Setup job description

In [2]:
# Import DAPI and other required libraries
from dapi import DSClient
import os
import json
from datetime import date

In [3]:
# Initialize DesignSafe client
ds = DSClient()

Authentication successful.
DatabaseAccessor initialized. Connections will be created on first access.


In [10]:
ds_path = "/home/jupyter/MyData/template-notebooks/tapis3/opensees/OpenSeesMP_multiMotion/DS_input"
print(f"DesignSafe path: {ds_path}")
input_uri = ds.files.translate_path_to_uri(ds_path)
print(f"Input URI: {input_uri}")

DesignSafe path: /home/jupyter/MyData/template-notebooks/tapis3/opensees/OpenSeesMP_multiMotion/DS_input
Translated '/home/jupyter/MyData/template-notebooks/tapis3/opensees/OpenSeesMP_multiMotion/DS_input' to 'tapis://designsafe.storage.default/kks32/template-notebooks/tapis3/opensees/OpenSeesMP_multiMotion/DS_input' using t.username
Input URI: tapis://designsafe.storage.default/kks32/template-notebooks/tapis3/opensees/OpenSeesMP_multiMotion/DS_input


In [8]:
# Job configuration parameters
jobname: str = "opensees-MP-multiMotion-dapi"
app_id: str = "opensees-mp-s3"
input_filename: str = "Main_multiMotion.tcl"
control_exec_Dir: str = "DS_input"  # Folder with files including input_filename
tacc_allocation: str = "ASC25049"  # MUST USE YOUR OWN ALLOCATION !!
control_nodeCount: int = 1
control_corespernode: int = 16
max_job_minutes: int = 60

In [13]:
# Generate job request dictionary using app defaults
job_dict = ds.jobs.generate_request(
    app_id=app_id,
    input_dir_uri=input_uri,
    script_filename=input_filename,
    max_minutes=max_job_minutes,
    allocation=tacc_allocation,
)

Generating job request for app 'opensees-mp-s3'...
Using App Details: opensees-mp-s3 vlatest
Placing script 'Main_multiMotion.tcl' in appArgs: 'Main Script'
Adding allocation: ASC25049
Job request dictionary generated successfully.


In [14]:
# Customize job settings
job_dict["name"] = jobname
job_dict["nodeCount"] = control_nodeCount
job_dict["coresPerNode"] = control_corespernode

print("Generated job request:")
print(json.dumps(job_dict, indent=2, default=str))

Generated job request:
{
  "name": "opensees-MP-multiMotion-dapi",
  "appId": "opensees-mp-s3",
  "appVersion": "latest",
  "description": "Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.",
  "execSystemId": "stampede3",
  "archiveSystemId": "stampede3",
  "archiveOnAppError": true,
  "execSystemLogicalQueue": "skx",
  "nodeCount": 1,
  "coresPerNode": 16,
  "maxMinutes": 60,
  "memoryMB": 192000,
  "isMpi": false,
  "tags": [],
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/template-notebooks/tapis3/opensees/OpenSeesMP_multiMotion/DS_input",
      "autoMountLocal": true,
      "targetPath": "inputDirectory"
    }
  ],
  "parameterSet": {
    "appArgs": [
      {
        "name": "Main Script",
        "arg": "Main_multiMotion.tcl"
      }
    ],
    "schedulerOptions": [
      {
        "name": "TACC Allocation",
        "arg": "-A AS

### Run job

In [15]:
# Submit job using dapi
submitted_job = ds.jobs.submit_request(job_dict)
print(f"Job launched with UUID: {submitted_job.uuid}")
print(
    "Can also check in DesignSafe portal under - Workspace > Tools & Application > Job Status"
)


--- Submitting Tapis Job Request ---
{
  "name": "opensees-MP-multiMotion-dapi",
  "appId": "opensees-mp-s3",
  "appVersion": "latest",
  "description": "Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.",
  "execSystemId": "stampede3",
  "archiveSystemId": "stampede3",
  "archiveOnAppError": true,
  "execSystemLogicalQueue": "skx",
  "nodeCount": 1,
  "coresPerNode": 16,
  "maxMinutes": 60,
  "memoryMB": 192000,
  "isMpi": false,
  "tags": [],
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/template-notebooks/tapis3/opensees/OpenSeesMP_multiMotion/DS_input",
      "autoMountLocal": true,
      "targetPath": "inputDirectory"
    }
  ],
  "parameterSet": {
    "appArgs": [
      {
        "name": "Main Script",
        "arg": "Main_multiMotion.tcl"
      }
    ],
    "schedulerOptions": [
      {
        "name": "TACC Allocation",
      

In [16]:
# Monitor job status using dapi
final_status = submitted_job.monitor(interval=15)
print(f"Job finished with status: {final_status}")

# Interpret job status
ds.jobs.interpret_status(final_status, submitted_job.uuid)

# Display runtime summary
submitted_job.print_runtime_summary(verbose=False)


Monitoring Job: 24938b27-854f-4a75-a687-fe21e8723b36-007
Job already in terminal state: FINISHED
Job finished with status: FINISHED
Job 24938b27-854f-4a75-a687-fe21e8723b36-007 completed successfully.

Runtime Summary
---------------
QUEUED  time: 00:03:57
RUNNING time: 00:01:01
TOTAL   time: 00:06:59
---------------


# Postprocess Results

### Identify job and archived location

In [17]:
# Get archive information using dapi
archive_uri = submitted_job.archive_uri
print(f"Archive URI: {archive_uri}")

# List archive contents
archive_files = ds.files.list(archive_uri)
print("\nArchive contents:")
for item in archive_files:
    print(f"- {item.name} ({item.type})")

Archive URI: tapis://stampede3/work2/05873/kks32/stampede3/tapis-jobs-archive/2025-06-05Z/opensees-MP-multiMotion-dapi-24938b27-854f-4a75-a687-fe21e8723b36-007
Listing files in system 'stampede3' at path 'work2/05873/kks32/stampede3/tapis-jobs-archive/2025-06-05Z/opensees-MP-multiMotion-dapi-24938b27-854f-4a75-a687-fe21e8723b36-007'...
Found 6 items.

Archive contents:
- inputDirectory (dir)
- opensees.zip (file)
- tapisjob.env (file)
- tapisjob.out (file)
- tapisjob.sh (file)
- tapisjob_app.sh (file)


### Go to archived folder

In [ ]:
# Download archive files to local directory for postprocessing
import tempfile
import shutil

# Create temporary directory for downloaded files
temp_dir = tempfile.mkdtemp(prefix="opensees_results_")
print(f"Downloading results to: {temp_dir}")

# Download the inputDirectory folder which contains results
input_dir_archive_uri = f"{archive_uri}/inputDirectory"
try:
    # List contents of inputDirectory in archive
    input_dir_files = ds.files.list(input_dir_archive_uri)
    print("\nFiles in inputDirectory:")
    for item in input_dir_files:
        print(f"- {item.name} ({item.type})")

    # Download all files from inputDirectory (excluding subdirectories)
    files_to_download = [item.name for item in input_dir_files if item.type == "file"]

    print(f"\nDownloading {len(files_to_download)} files...")
    successful_downloads = []

    for filename in files_to_download:
        try:
            file_uri = f"{input_dir_archive_uri}/{filename}"
            local_path = os.path.join(temp_dir, filename)

            # Try downloading with the ds.files.download method
            try:
                ds.files.download(file_uri, local_path)
                print(f"Downloaded: {filename}")
                successful_downloads.append(filename)
            except Exception as download_error:
                print(f"Standard download failed for {filename}: {download_error}")

                # Try alternative download approach using get_file_content
                try:
                    content = ds.files.get_file_content(file_uri)
                    with open(local_path, "wb") as f:
                        if hasattr(content, "read"):
                            shutil.copyfileobj(content, f)
                        else:
                            f.write(content)
                    print(f"Downloaded (alternative method): {filename}")
                    successful_downloads.append(filename)
                except Exception as alt_error:
                    print(
                        f"Alternative download also failed for {filename}: {alt_error}"
                    )

        except Exception as e:
            print(f"Could not download {filename}: {e}")

    print(
        f"\nSuccessfully downloaded {len(successful_downloads)} out of {len(files_to_download)} files"
    )

except Exception as e:
    print(f"Error accessing archive: {e}")

# Change to the temporary directory for postprocessing
os.chdir(temp_dir)
print(f"\nChanged to directory: {os.getcwd()}")
print("Local files:")
for f in sorted(os.listdir(".")):
    print(f"- {f}")

### Import python libraries

In [20]:
!pip3 install matplotlib  # Install matplotlib for plotting

  Using cached contourpy-1.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pillow-11.2.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (8.9 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 26.4 MB/s eta 0:00:00a 0:00:01
Using cached contourpy-1.3.2-cp311-cp311-macosx_11_0_arm64.whl (254 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 23.6 MB/s eta 0:00:00
Using cached kiwisolver-1.4.8-cp311-cp311-macosx_11_0_arm64.whl (65 kB)
Using cached pillow-11.2.1-cp311-cp311-macosx_11_0_arm64.whl (3.0 MB)
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [21]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Plot acceleration response spectra

Plot acceleration response spectra on log-linear scale

In [ ]:
# Define response spectra function inline
def resp_spectra(a, time, nstep):
    """
    This function builds response spectra from acceleration time history,
    a should be a numpy array,T and nStep should be integers.
    """
    # add initial zero value to acceleration and change units
    a = np.insert(a, 0, 0)
    # number of periods at which spectral values are to be computed
    nperiod = 100
    # define range of considered periods by power of 10
    minpower = -3.0
    maxpower = 1.0
    # create vector of considered periods
    p = np.logspace(minpower, maxpower, nperiod)
    # incremental circular frequency
    dw = 2.0 * np.pi / time
    # vector of circular freq
    w = np.arange(0, (nstep + 1) * dw, dw)
    # fast fourier transform of acceleration
    afft = np.fft.fft(a)
    # arbitrary stiffness value
    k = 1000.0
    # damping ratio
    damp = 0.05
    umax = np.zeros(nperiod)
    vmax = np.zeros(nperiod)
    amax = np.zeros(nperiod)
    # loop to compute spectral values at each period
    for j in range(0, nperiod):
        # compute mass and dashpot coeff to produce desired periods
        m = ((p[j] / (2 * np.pi)) ** 2) * k
        c = 2 * damp * (k * m) ** 0.5
        h = np.zeros(nstep + 2, dtype=complex)
        # compute transfer function
        for l in range(0, int(nstep / 2 + 1)):
            h[l] = 1.0 / (-m * w[l] * w[l] + 1j * c * w[l] + k)
            # mirror image of transfer function
            h[nstep + 1 - l] = np.conj(h[l])

        # compute displacement in frequency domain using transfer function
        qfft = -m * afft
        u = np.zeros(nstep + 1, dtype=complex)
        for l in range(0, nstep + 1):
            u[l] = h[l] * qfft[l]

        # compute displacement in time domain (ignore imaginary part)
        utime = np.real(np.fft.ifft(u))

        # spectral displacement, velocity, and acceleration
        umax[j] = np.max(np.abs(utime))
        vmax[j] = (2 * np.pi / p[j]) * umax[j]
        amax[j] = (2 * np.pi / p[j]) * vmax[j]

    return p, umax, vmax, amax


# Define plot_acc function inline
def plot_acc():
    """
    Plot acceleration time history and response spectra
    """
    plt.figure(figsize=(12, 8))

    # Check for available acceleration files
    acc_files = []
    for motion in ["motion1", "motion2"]:
        for profile in ["A", "B", "C", "D"]:
            filename = f"Profile{profile}_acc{motion}.out"
            if os.path.exists(filename):
                acc_files.append((motion, profile, filename))

    if not acc_files:
        print("No acceleration output files found.")
        print("Available files in current directory:")
        for f in os.listdir("."):
            if f.endswith(".out"):
                print(f"- {f}")
        return

    # Create subplot for response spectra
    plt.subplot(2, 1, 1)

    colors = ["b", "r", "g", "m"]
    motion_styles = {"motion1": "-", "motion2": "--"}

    for i, (motion, profile, filename) in enumerate(acc_files):
        try:
            acc = np.loadtxt(filename)
            if acc.size > 0 and acc.ndim >= 2:
                [p, umax, vmax, amax] = resp_spectra(
                    acc[:, -1], acc[-1, 0], acc.shape[0]
                )
                color = colors[ord(profile) - ord("A")]
                style = motion_styles[motion]
                plt.semilogx(
                    p,
                    amax,
                    color=color,
                    linestyle=style,
                    label=f"Profile {profile}, {motion}",
                )
        except Exception as e:
            print(f"Error processing {filename}: {e}")

    plt.ylabel("$S_a (g)$")
    plt.xlabel("$Period (s)$")
    plt.title("Acceleration Response Spectra")
    plt.grid(True, alpha=0.3)
    plt.legend()

    # Create subplot for time history (if data available)
    plt.subplot(2, 1, 2)
    for i, (motion, profile, filename) in enumerate(
        acc_files[:4]
    ):  # Show first 4 files
        try:
            acc = np.loadtxt(filename)
            if acc.size > 0 and acc.ndim >= 2:
                time = np.linspace(0, acc[-1, 0], acc.shape[0])
                color = colors[i % len(colors)]
                plt.plot(
                    time,
                    acc[:, -1],
                    color=color,
                    alpha=0.7,
                    label=f"Profile {profile}, {motion}",
                )
        except Exception as e:
            print(f"Error plotting time history for {filename}: {e}")

    plt.ylabel("Acceleration (g)")
    plt.xlabel("Time (s)")
    plt.title("Acceleration Time History")
    plt.grid(True, alpha=0.3)
    plt.legend()

    plt.tight_layout()
    plt.show()


# Execute the plotting function
plot_acc()